In [1]:
# TDO COMBINE TRAIN HISTORY DATA TO ONE FILE PER DAY
# TODO CALC TRAVALED PASSENGERN FROM TO FOR EACH DAY FOR EVERY STATION TO ONE! NEXT STATION

In [2]:
import os
import pandas as pd
from pathlib import Path
from datetime import datetime
import json


cwd = os.getcwd()
dep_table_path = os.path.join(cwd, 'generated/departure_tables_raw')
dep_table_path

'/home/jovyan/work/generated/departure_tables_raw'

In [3]:
raw_departure_tables = []
departure_tables_dates = []
departure_tables_dates_ts = []
# LOAD ALL CSV FILES INTO ONE BIG CHUNK
files = [ f for f in os.listdir( dep_table_path)] #list comprehension version.

for f in files:
    if(Path(f).suffix == ".csv"):
        filename = str(Path(f).stem)
        if '_' in filename and len(filename.split('_')) == 4:
            # THE GOAL IS TO COMBINE ALL TRAIN DEPARTURES FROM THE SAME DAY
            # SO THE FILE TIMESTAMP IN ITS NAME CORRESPDS TO THE DATE OF RECORD = TRAIN DEPARTURES
            # BY ADDING THE NEW DATE TO THE PD DF ITS IS POSSIBLE TO SORT FOR DATES LATER ON
            timestamp = int(filename.split('_')[3])
            date = datetime.fromtimestamp(timestamp)
            # THIS WILL BE ADDED TO THE DF AS NEW COLUMN AFTER LOADING
            combined_day = str(date.day).zfill(2) + str(date.month).zfill(2) + str(date.year).zfill(2)
            departure_tables_dates.append(combined_day)
            departure_tables_dates_ts.append(timestamp)
            # LOAD FILE TO DF
            abs_path = os.path.join(dep_table_path,f)
            df = pd.read_csv(abs_path,sep=';',encoding="utf-8",dtype = str)
            
            # ADD NEW COLUMNS
            df['date'] = pd.Series([date for x in range(len(df.index))])
            df['combined_date'] = pd.Series([combined_day for x in range(len(df.index))])
            
            raw_departure_tables.append(df)


            
# CREATE DATASET FROM DATE VALUES
df_departure_tables_dates = pd.DataFrame(data={'recorded_days':departure_tables_dates})
            
# AFER LOADING ALL RAW FILE IN CREATE ONE GIANT DATASET
departures_combined = pd.concat(raw_departure_tables)
departures_combined.head(5)


,current_station_departure_time,current_station_name,type,name,line,number,current_station,stops,max_capacity,date,combined_date
0,2022-01-02T12:02:00.000Z,Bayreuth Hbf,RE,RE 38,38,59306,8000028,"8000028%2022-01-02T12:02:00.000Z%0,8005895,800...",602,2022-01-02 11:46:01,02012022
1,2022-01-02T12:15:00.000Z,Bayreuth Hbf,RE,RE 30,30,3410,8000028,"8000028%2022-01-02T12:15:00.000Z%0,8001348,800...",602,2022-01-02 11:46:01,02012022
2,2022-01-02T12:32:00.000Z,Bayreuth Hbf,RE,RE 30,30,3087,8000028,"8000028%2022-01-02T12:34:00.000Z%2,8004126,800...",602,2022-01-02 11:46:01,02012022
3,2022-01-02T13:01:00.000Z,Bayreuth Hbf,RE,RE 32,32,3472,8000028,"8000028%2022-01-02T13:01:00.000Z%0,8001348,800...",602,2022-01-02 11:46:01,02012022
4,2022-01-02T13:02:00.000Z,Bayreuth Hbf,RE,RE 32,32,3487,8000028,"8000028%2022-01-02T13:03:00.000Z%1,8005895,800...",602,2022-01-02 11:46:01,02012022


In [4]:
# REMOVE DUPLICATE TRAINS TRAIN_ID AND START_STATION
departures_combined = departures_combined.drop_duplicates(['number','current_station_name','combined_date'],keep= 'last')
departures_combined.head(5)


,current_station_departure_time,current_station_name,type,name,line,number,current_station,stops,max_capacity,date,combined_date
2572,2022-01-14T00:34:00.000Z,Raisdorf,RB,RB 84,84,11138,8004924,"8004924%2022-01-14T00:34:00.000Z%0,8003477,800...",426,2022-01-14 00:13:42,14012022
4441,2022-01-14T00:18:00.000Z,Unterheckenhofen,RB,RB 16,16,59229,8005995,"8005995%2022-01-14T00:20:00.000Z%2,8002237,800...",426,2022-01-14 00:13:42,14012022
4800,2022-01-14T00:16:00.000Z,Wackershofen,RB,RB 83,83,23439,8006135,"8006135%2022-01-14T00:18:00.000Z%2,8005449,800...",426,2022-01-14 00:13:42,14012022
4886,2022-01-14T00:09:00.000Z,Waldenburg(Württ),RB,RB 83,83,23439,8006154,"8006154%2022-01-14T00:10:00.000Z%1,8006135,800...",426,2022-01-14 00:13:42,14012022
9478,2022-01-14T00:25:00.000Z,Erkner,RE,RE 1,1,3153,8013477,"8013477%2022-01-14T00:25:00.000Z%0,8011533,801...",602,2022-01-14 00:13:42,14012022


In [5]:
# LETS TEST THE DATA
# GET ALL TRAIN DEPARTURES FROM Bayreuth Hbf [8000028] @ 29.11.2021
# RV + FV + DB ONLY
tmp = departures_combined.loc[departures_combined['current_station'] == str(8000028)]
tmp = tmp.loc[tmp['combined_date'] == departure_tables_dates[0]]
tmp = tmp.reset_index()
tmp.head(100)

,index,current_station_departure_time,current_station_name,type,name,line,number,current_station,stops,max_capacity,date,combined_date
0,14,2022-01-03T07:29:00.000Z,Bayreuth Hbf,RE,RE 30,30,3080,8000028,"8000028%2022-01-03T07:29:00.000Z%,8004759,8004...",602,2022-01-02 20:15:53,02012022
1,16,2022-01-03T08:03:00.000Z,Bayreuth Hbf,RE,RE 32,32,3456,8000028,"8000028%2022-01-03T08:03:00.000Z%,8004759,8002...",602,2022-01-02 20:15:53,02012022
2,2,2022-01-02T05:05:00.000Z,Bayreuth Hbf,RE,RE 30,30,3452,8000028,"8000028%2022-01-02T05:05:00.000Z%,8001348,8000...",602,2022-01-02 01:49:12,02012022
3,5,2022-01-02T06:32:00.000Z,Bayreuth Hbf,RE,RE 30,30,3097,8000028,"8000028%2022-01-02T06:32:00.000Z%,8004126,8002...",602,2022-01-02 01:49:12,02012022
4,8,2022-01-02T08:02:00.000Z,Bayreuth Hbf,RE,RE 38,38,59302,8000028,"8000028%2022-01-02T08:02:00.000Z%,8005895,8000...",602,2022-01-02 01:49:12,02012022
5,9,2022-01-02T08:15:00.000Z,Bayreuth Hbf,RE,RE 30,30,3406,8000028,"8000028%2022-01-02T08:15:00.000Z%,8001348,8000...",602,2022-01-02 01:49:12,02012022
6,10,2022-01-02T08:32:00.000Z,Bayreuth Hbf,RE,RE 30,30,3083,8000028,"8000028%2022-01-02T08:32:00.000Z%,8004126,8002...",602,2022-01-02 01:49:12,02012022
7,11,2022-01-02T09:01:00.000Z,Bayreuth Hbf,RE,RE 32,32,3448,8000028,"8000028%2022-01-02T09:01:00.000Z%,8001348,8004...",602,2022-01-02 01:49:12,02012022
8,12,2022-01-02T09:02:00.000Z,Bayreuth Hbf,RE,RE 32,32,3483,8000028,"8000028%2022-01-02T09:02:00.000Z%,8000267,8003...",602,2022-01-02 01:49:12,02012022
9,13,2022-01-02T09:29:00.000Z,Bayreuth Hbf,RE,RE 30,30,3082,8000028,"8000028%2022-01-02T09:29:00.000Z%,8004759,8004...",602,2022-01-02 01:49:12,02012022


In [6]:
# COUNT THE UNIQUE TRAINS
#THE TRAIN_ID IS UNIQUE FOR EACH TRAIN AND IDENTIFIES THE TRAIN OVER ITS COMPLETE COURSE
# ONE UNIQUE ID = ONE ZUGLAUF
# FROM START TO END-STATION
tmp = departures_combined.loc[departures_combined['combined_date'] == departure_tables_dates[0]]
tmp['number'].nunique()

8893

In [7]:
# REMOVE DUPLICATES IN THE DATES
df_departure_tables_dates = df_departure_tables_dates.drop_duplicates()

In [8]:
# SAVE RESULTS AS CSV
departures_combined.to_csv("./generated/4_departure_table.csv",sep=';',encoding="utf-8",index=False)
df_departure_tables_dates.to_csv("./generated/4_departure_table_day_list.csv",sep=';',encoding="utf-8",index=False)

In [9]:
# SAVE RESULT AS JSON FOR WEBSERVER
df_departure_tables_dates

,recorded_days
0,02012022
1,10012022
2,09012022
3,12012022
5,03012022
6,14012022
7,08012022
8,13012022
10,30122021
11,06012022


In [10]:
jdict = {
    'recorded_dates': df_departure_tables_dates['recorded_days'].values.tolist()
}
jdict_string = json.dumps(jdict)
with open("./generated/4_departure_table_day_list.json", "w") as outfile:
    json.dump(jdict_string, outfile)

In [11]:
# CREATE 4_station_train_departure_lookuptable
# THIS LOOKUP TABLE CONTAINS AS A KEY THE STATION ID AND AS VALUE
# ALL TRAINS THAT DEPARTURES AT THIS STATOION

# LOAD SELECTED STATION
stations = pd.read_csv("./generated/0_db_station_lk.csv",sep=';',encoding="utf-8")
stations.head(5)

,db_station_name,rki_landkreisname,rki_bezeichner,db_station_id,rki_ags
0,Bayreuth Hbf,Bayreuth,Kreisfreie Stadt,8000028,9462
1,Brackwede,Bielefeld,Kreisfreie Stadt,8000048,5711
2,Kaufering,Landsberg am Lech,Landkreis,8000195,9181
3,Weiden(Oberpf),Weiden i.d. OPf.,Kreisfreie Stadt,8000204,9363
4,Landshut(Bay)Hbf,Landshut,Kreisfreie Stadt,8000217,9261


In [12]:
station_ids = stations['db_station_id']
station_ids.head(5)

0    8000028
1    8000048
2    8000195
3    8000204
4    8000217
Name: db_station_id, dtype: int64

In [13]:
# FETCH EACH STATION
start_train_lut = {}
for station in station_ids:

    # GET ALL TRAINS CURRENTLY DEPATURING ON THE SELECTED STATION
    # REMOVE ALL DUPLICATES => UNIUQ TRAIN NAMES
    tmptt_lut = departures_combined.loc[departures_combined['current_station'].isin([str(station)])]
    tmptt_lut = tmptt_lut.drop_duplicates(subset=['name'], keep='last')
    
    
    
    start_train_lut[str(station)] = {'stops': tmptt_lut['stops'].values.tolist(),'trains': tmptt_lut['name'].values.tolist(), 'capacity': tmptt_lut['max_capacity'].values.tolist()}
start_train_lut

{'8000028': {'stops': ['8000028%2022-01-12T16:02:00.000Z%,8005895,8000267,8006017,8003476,8001289,8000228,',
   '8000028%2022-01-12T17:03:00.000Z%,8005895,8000267,8003476,8003813,8001289,8000228,',
   '8000028%2022-01-12T17:30:00.000Z%,8004759,8004284,8000284,'],
  'trains': ['RE 38', 'RE 32', 'RE 30'],
  'capacity': ['602', '602', '602']},
 '8000048': {'stops': ['8000048%2022-01-01T14:27:00.000Z%,8000036,',
   '8000048%2022-01-13T22:57:00.000Z%0,8000036,',
   '8000048%2022-01-14T06:21:00.000Z%,8000036,8000162,8000233,8000732,8004868,8000252,8004777,8000335,8000279,',
   '8000048%2022-01-12T19:15:00.000Z%,8002461,8000315,8002801,8001335,8000846,8006206,8006207,8005834,8000263,',
   '8000048%2022-01-12T19:30:00.000Z%,8003096,8002461,8000315,8004626,8004251,8000441,8002680,8000149,8001045,8003986,8001558,8005102,8002841,8000263,'],
  'trains': ['RB 89', 'RE 6', 'RE 78', 'RB 67', 'RB 69'],
  'capacity': ['426', '602', '602', '426', '426']},
 '8000195': {'stops': ['8000195%2022-01-13T09:03

In [14]:
# SAVE AS JSON
import codecs
_tmp = json.dumps(start_train_lut, indent=4, ensure_ascii=False)
with codecs.open("./generated/4_station_train_departure_lookuptable.json", 'w', encoding='utf-8') as outfile:
    outfile.write(_tmp)

In [16]:
# FOR THE DATEPICKER ON THE VISUALISATION SITE WE NEED A LIST OF DATES FROM THE DONWLOADED COVID DATA
cwd = os.getcwd()
covid_table_path = os.path.join(cwd, 'generated/corona_histroy')
covid_table_path

'/home/jovyan/work/generated/corona_histroy'

In [91]:
covid_tables_dates = []
covid_tables_dates_lut = {}
test= [] # TODO REMOVE
# LOAD ALL CSV FILES INTO ONE BIG CHUNK
files = [ f for f in os.listdir( covid_table_path)]
pd_cv = None
for f in files:
    if(Path(f).suffix == ".csv"):
        filename = str(Path(f).stem)
        sp = filename.split('_')
        if '_' in filename and len(sp) == 2:
            ts= sp[1]
            dt_object = datetime.fromtimestamp(int(ts))
            day = dt_object.day
            if day < 10:
                day = "0" + str(day)
            # HAHA BAD
            month = dt_object.month
            if month < 10:
                month = "0" + str(month)
            
            year = dt_object.year
            if year < 100:
                year = "20" + year
            # TODO REMOVE
            if (str(day)+str(month)+str(year)) in test:
                continue
            
            
            
            # INCLUDE COVID DATA FROM FILE DIRECTLY AS LOOKUP TABLE
            # ITS LATER BE USED TO COLOR THE MAP
            covid_tables_dates_lut[str(day)+str(month)+str(year)] = None #MAYBE FOR LATER
            pd_cv = pd.read_csv(os.path.join(covid_table_path,f),sep=';',encoding="utf-8")
            # SOMEL UT REORDERING
            # dict(zip(pd_cv['rki_ags'], zip(df["population"], df_['cases_100k'])))
            covid_tables_dates_lut[str(day)+str(month)+str(year)] = {
                'population': pd_cv.set_index("rki_ags")["population"].to_dict(),
                'cases_100k': pd_cv.set_index("rki_ags")["cases_100k"].to_dict(),
                'incidence': pd_cv.set_index("rki_ags")["incidence"].to_dict()
            }

            #pd_cv.to_dict('records')
            
            
            # SAVE DATES IN ARRAY FOR EASY LIST VIEW IN HTML
            covid_tables_dates.append({'ts':ts,'sys':str(day)+str(month)+str(year),'readable':str(day)+'.'+str(month)+'.'+str(year)})
            test.append(str(day)+str(month)+str(year)) # TODO REMOVE
covid_tables_dates_lut['recorded_dates'] = covid_tables_dates
covid_tables_dates_lut


{'29112021': {'population': {9462: 74048,
   5711: 333509,
   9181: 121019,
   9363: 42535,
   9261: 73065,
   5758: 250547,
   3354: 48503,
   9361: 42052,
   9661: 70858,
   8211: 55449,
   3459: 359471,
   5314: 330579,
   4011: 566573,
   4012: 113557,
   9463: 40842,
   5166: 298536,
   3355: 184235,
   1004: 79905,
   3252: 148580,
   9562: 112385,
   5970: 275491,
   3361: 137574,
   5512: 117388,
   7311: 48750,
   8311: 230940,
   9172: 106327,
   5513: 259105,
   3257: 158406,
   5915: 178967,
   8121: 126458,
   5916: 156940,
   9464: 45173,
   9161: 136952,
   9761: 295830,
   5754: 364818,
   9572: 138105,
   8212: 308436,
   7111: 113388,
   3154: 91518,
   5114: 226844,
   7313: 46685,
   6531: 271667,
   7133: 158746,
   5554: 371898,
   5316: 163905,
   3256: 121645,
   7314: 172557,
   5558: 220712,
   7315: 217123,
   9675: 91696,
   9376: 148477,
   9673: 79510,
   5358: 265140,
   9771: 135024,
   8327: 141682,
   5117: 170921,
   6432: 297701,
   3353: 256016,
   

In [90]:

_tmp = json.dumps(covid_tables_dates_lut, indent=4, ensure_ascii=False)
with codecs.open("./generated/4_downloaded_covid_data_dates.json", 'w', encoding='utf-8') as outfile:
    outfile.write(_tmp)